# Settings

In [ ]:
# set to false if working on local machine
onColab = True

epochs = 200
batch_size = 64

# set to true if you need to unzip the data
doUnzipData = False

# set to false to delete test folder and get ready to submit!
# set to true to use/create test folder
isTest = True

# decide which model to train 
# You can also set them all to true. They will be all trained
baseline = False
resnet50 = False
vgg16 = True

# PATH_TO_CHECKPOINTS is a folder which contains/will contain the checkpoints for the models. It is not necessary that it already exists
PATH_TO_CHECKPOINTS = './checkpoints/'

# Init
We initialize google drive, import the libraries, and initialize whatever *global* variable we need.

## Google drive access

In [ ]:
if onColab:
  from google.colab import drive
  drive.mount('/gdrive')

In [ ]:
if onColab:
  cd /gdrive/MyDrive/University/ANN/CHALLENGE1/
  !ls

## Importing libs

In [ ]:
import os
import time
import scipy
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
!pip install visualkeras
import visualkeras

## Setting seed

In [ ]:
seed = 20

In [ ]:
#@title init seed everywhere
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Some parameters init

In [ ]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

baseline_path_to_ckpt = PATH_TO_CHECKPOINTS + 'baseline/ckpts/'
resnet50_path_to_ckpt = PATH_TO_CHECKPOINTS + 'resnet50/ckpts/'
vgg16_path_to_ckpt = PATH_TO_CHECKPOINTS + 'vgg16/ckpts/'

dataset_dir = 'training'
dataset_path = './training'

img_h, img_w = (256, 256)
input_shape = (img_h, img_w, 3)

# check total number of files. It must be 17728 if no testset
def countInTraining():
  count = 0
  for i in range(len(labels)):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    for j in range(len(class_imgs)):
      count += 1
  print("In training: ", count)
  return count

## Unzip dataset

In [ ]:
#@title Run this cell to unzip data
if doUnzipData:
  !unzip dataset.zip

In [ ]:
#@title Plot example images from dataset
if doUnzipData:
  num_row = len(labels)//2
  num_col = len(labels)//num_row
  fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
  for i in range(len(labels)):
    if i < len(labels):
      class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
      class_img = class_imgs[0]
      print(class_imgs)
      # print(class_img)
      img = Image.open('{}/{}/{}'.format(dataset_dir, labels[i], class_img))
      ax = axes[i//num_col, i%num_col]
      ax.imshow(np.array(img))
      ax.set_title('{}'.format(labels[i]))
  plt.tight_layout()
  plt.show()

## Only if evaluating the performance
If test folder is not present we create one. 

For each class type we generate a folder and we move 1/100 images of that type to that folder

In [ ]:
test_path = "./test"
if isTest and not os.path.exists(test_path):
  import shutil
  os.mkdir(test_path)
  for i in range(len(labels)):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    # In this way we always get the same data. 
    # TODO: randomize using seed the data we take
    class_imgs.sort()
    # put in test 1/100 of the pictures of one folder
    for j in range(len(class_imgs) // 100):
      class_img = class_imgs[j]
      dest_dir = './test/' + labels[i] + '/'
      if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)
      shutil.move('{}/{}/'.format(dataset_dir, labels[i]) + class_img, dest_dir)

countInTraining()

## If we want to submit the model
We do not generate test folder. 

If test folder is present we move the images back into the train folder and delete the test folder

In [ ]:
test_path = "./test"
if os.path.exists(test_path) and not isTest:
  import shutil
  for i in range(len(labels)):
    class_imgs = next(os.walk('{}/{}/'.format(test_path, labels[i])))[2]
    # put back in training the picture in test
    for j in range(len(class_imgs)):
      class_img = class_imgs[j]
      dest_dir = './training/' + labels[i] + '/'
      shutil.move('{}/{}/'.format(test_path, labels[i]) + class_img, dest_dir)
  shutil.rmtree(test_path)

  if countInTraining() != 17728:
    print("CAREFUL: some images could have been deleted")

countInTraining()

# Import data, preprocess, useful functions

## Import data

In [ ]:
#@title Another possible way to load the dataset
# # One possible way to load the dataset. I use another which was used also in class, using ImageDataGenerator
# train_ds = tf.keras.utils.image_dataset_from_directory(
#   dataset_path,
#   validation_split=0.2,
#   subset="training",
#   seed=seed,
#   image_size=(img_h, img_w),
#   batch_size=batch_size)

# # Check sizes
# # train_ds is divided in batch of batch_size with images with shape (img_h, img_w, n_channels)
# for image_batch, labels_batch in train_ds:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break

# val_ds = tf.keras.utils.image_dataset_from_directory(
#   dataset_path,
#   validation_split=0.2,
#   subset="validation",
#   seed=seed,
#   image_size=(img_h, img_w),
#   batch_size=batch_size)

# # just trying to print some images in the train_ds
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(len(train_ds.class_names)):
#     ax = plt.subplot(4, 4, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(train_ds.class_names[labels[i]])
#     plt.axis("off")

In [ ]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.

# Create an instance of ImageDataGenerator for training and validation
# We also apply data augmentation
def load_data(dontUseFun=True, fun=None):
  image_generator = ImageDataGenerator(preprocessing_function = fun, validation_split=0.2)  
  if dontUseFun:
    image_generator = ImageDataGenerator(rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True, 
                                          fill_mode='reflect',
                                          rescale=1./255, 
                                          validation_split=0.2)  


  # Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
  train_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                subset='training',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)

  valid_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                subset='validation',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  test_gen = None
  if isTest:
    test_image_gen = ImageDataGenerator()
    if dontUseFun:
      test_image_gen = ImageDataGenerator(rescale=1./255)
    test_gen = test_image_gen.flow_from_directory(directory=test_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  return train_gen, valid_gen, test_gen
train_gen, valid_gen, test_gen = load_data()

In [ ]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  print("Class name:", labels[target_idx])
  fig = plt.figure(figsize=(6, 4))
  gen = ImageDataGenerator()  
  plt.imshow(np.uint8(image * 255))

  return batch

# Get a sample from dataset and show info
_ = get_next_batch(train_gen)

## Model Comparison
Here we put functions to plot models against other models

In [ ]:
def compareModels(history1, h1lbl, history2=None, h2lbl=''):
  '''
  Plot history1 against history2. 
  If history2 is None then Plot twice history1 (it'll be overlapped)
  h1lbl and h2lbl are the labels of the two histories
  '''
  if history2 == None: 
    history2 = history1
    h2lbl = h1lbl
  # Plot the training
  plt.figure(figsize=(15,5))
  plt.plot(history1['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
  plt.plot(history1['val_loss'], label=h1lbl, alpha=.8, color='#ff7f0e')
  plt.plot(history2['loss'], alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(history2['val_loss'], label=h2lbl, alpha=.8, color='#4D61E2')
  plt.legend(loc='upper left')
  plt.title('Categorical Crossentropy')
  plt.grid(alpha=.3)

  plt.figure(figsize=(15,5))
  plt.plot(history1['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
  plt.plot(history1['val_accuracy'], label=h1lbl, alpha=.8, color='#ff7f0e')
  plt.plot(history2['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(history2['val_accuracy'], label=h2lbl, alpha=.8, color='#4D61E2')
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=.3)

  plt.show()

## Callbacks & checkpoints
Tired of colab disconnections? Me too, let's save checkpoints while training :)

Here there is also early stopping

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name, ckpt_path=PATH_TO_CHECKPOINTS, overridePrev = True):

  if not os.path.exists(ckpt_path):
      os.makedirs(ckpt_path)
  now = ''
  if not overridePrev: now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(ckpt_path, model_name + '_' + str(now))
  if now == '': exp_dir = os.path.join(ckpt_path, model_name)

  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch. 
                                                                           # We use early stopping, thus, in this way we can save both the last and the best
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  return callbacks

def callbacks_with_es(model_name, ckpt_path=PATH_TO_CHECKPOINTS, overridePrev = True):
  callbacks = create_folders_and_callbacks(model_name, ckpt_path, overridePrev)
  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)
  return callbacks

def retrieveCheckpoint(path_to_ckps):
  try:
    return tf.keras.models.load_model(path_to_ckps).evaluate(test_gen, return_dict=True)
  except:
    # TODO: better exception handling
    return None

# Baseline model
I just use the model proposed in class

In [ ]:
# Model used for the exercise:
# (Conv + ReLU + MaxPool) x 5 + FC x 2
def build_baseline_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv5)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
#@title This is declared above with changes (Keep it also here commented out)

# # Utility function to create folders and callbacks for training
# from datetime import datetime

# def create_folders_and_callbacks(model_name):

#   exps_dir = os.path.join('experiments')
#   if not os.path.exists(exps_dir):
#       os.makedirs(exps_dir)

#   now = datetime.now().strftime('%b%d_%H-%M-%S')

#   exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
#   if not os.path.exists(exp_dir):
#       os.makedirs(exp_dir)
      
#   callbacks = []

#   # Model checkpoint
#   # ----------------
#   ckpt_dir = os.path.join(exp_dir, 'ckpts')
#   if not os.path.exists(ckpt_dir):
#       os.makedirs(ckpt_dir)

#   ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
#                                                      save_weights_only=False, # True to save only weights
#                                                      save_best_only=False) # True to save only the best epoch. 
#                                                                            # We use early stopping, thus, in this way we can save both the last and the best
#   callbacks.append(ckpt_callback)

#   # Visualize Learning on Tensorboard
#   # ---------------------------------
#   tb_dir = os.path.join(exp_dir, 'tb_logs')
#   if not os.path.exists(tb_dir):
#       os.makedirs(tb_dir)
      
#   # By default shows losses and metrics for both training and validation
#   tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
#                                                profile_batch=0,
#                                                histogram_freq=1)  # if > 0 (epochs) shows weights histograms
#   callbacks.append(tb_callback)

#   # Early Stopping
#   # --------------
#   es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#   callbacks.append(es_callback)

#   return callbacks

In [ ]:
# Build model (for data augmentation training)
if baseline:
  model = retrieveCheckpoint(baseline_path_to_ckpt)
  if model == None:
    model = build_baseline_model(input_shape)
  model.summary()

In [ ]:
tf.get_logger().setLevel('WARNING') #  if you want to suppress only INFOs
# tf.get_logger().setLevel('ERROR') #  if you want to suppress both WARNINGs and INFOs

# Create folders and callbacks and fit
aug_callbacks = callbacks_with_es(model_name='baseline')

if baseline:
  # Train the model
  history = model.fit(
      x = train_gen,
      epochs = epochs,
      validation_data = valid_gen,
      callbacks = aug_callbacks,
  ).history

In [ ]:
# Save best epoch model
if baseline:
  model.save("experimets/baseline_best")

# Resnet50
Transfer learning on Resnet50

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

# reload data without preprocessing
train_gen, valid_gen, test_gen = load_data(False, preprocess_input)

In [ ]:
supernet = tfk.applications.ResNet50(
    include_top=False, # we remove the classifier
    weights="imagenet",
    input_shape=(img_h * 2, img_w * 2, 3)
)
# supernet.summary()
# tfk.utils.plot_model(supernet)

In [ ]:
supernet.trainable = False # we want to train only the classifier, not the feature extractor

# Now we use the supernet as layer of our network
inputs = tfk.Input(shape=(img_h, img_w, 3))
x = tfkl.Resizing(img_h * 2, img_w * 2, interpolation="bicubic")(inputs)
x = supernet(x)

# Now we build our classifier (which will be actually trained)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    len(labels), 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

# Look if there is a checkpoint
res_model = retrieveCheckpoint(resnet50_path_to_ckpt)
if res_model == None:
  # Connect input and output through the Model class
  res_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model
  res_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

res_model.summary()

In [ ]:
if resnet50:
  callbacks = create_folders_and_callbacks(model_name='resnet50')
  callbacks.append(tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True))
  res_history = res_model.fit(
      x = train_gen,
      batch_size = batch_size,
      epochs = epochs,
      validation_data = valid_gen,
      callbacks = callbacks
  ).history

In [ ]:
if resnet50:
  res_model.save("./resnet50_best")

In [ ]:
if resnet50:
    compareModels(res_history, 'resnet50')

# VGG16
Transfer learning with fine tuning on VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# reload data without preprocessing
train_gen, valid_gen, test_gen = load_data(False, preprocess_input)

In [ ]:
supernet = tfk.applications.VGG16(
    include_top=False, # we remove the classifier
    weights="imagenet",
    input_shape=(img_h * 2, img_w * 2, 3)
)
# supernet.summary()
# tfk.utils.plot_model(supernet)

In [ ]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(supernet.layers[:14]):
  layer.trainable=False
for i, layer in enumerate(supernet.layers):
   print(i, layer.name, layer.trainable)
# supernet.summary()

# Now we use the supernet as layer of our network
inputs = tfk.Input(shape=(img_h, img_w, 3))
x = tfkl.Resizing(img_h * 2, img_w * 2, interpolation="bicubic")(inputs)
x = supernet(x)

# Now we build our classifier (which will be actually trained)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    len(labels), 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

# Look if there is a checkpoint
vgg_model = retrieveCheckpoint(vgg16_path_to_ckpt)
if vgg_model == None:
  # Connect input and output through the Model class
  vgg_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model
  vgg_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

vgg_model.summary()

In [ ]:
if vgg16:
  callbacks = create_folders_and_callbacks(model_name='vgg16')
  callbacks.append(tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True))
  vgg_history = vgg_model.fit(
      x = train_gen,
      batch_size = batch_size,
      epochs = epochs,
      validation_data = valid_gen,
      callbacks = callbacks
  ).history

In [ ]:
if vgg16:
  vgg_model.save("./vgg16_best")

In [ ]:

if vgg16:
    compareModels(vgg_history, 'vgg16')